cargamos los datos de la carpeta processeds que se supone que ya hayan sido creados previamente por el proceso de etl

entrenamos los modelos con esos datos y los guardamos en la carpeta models como archivos .pkl en el caso de los modelos de ML y como archivos .h5 en el caso de los modelos de DL